In [1]:
names = ['Socrates', 'Archimedes', 'Plato', 'Aristotle']
names.sort(key=len)
print(names)

['Plato', 'Socrates', 'Aristotle', 'Archimedes']


### *first-class* function: Functions work and methods can be passed around and referenced like any other value in the langeage

In [2]:
#define a hook that logs each time a key is missing and returns 0 for the default value
def log_missing():
    print('Key added')
    return 0

In [6]:
from collections import defaultdict

current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9)
]
result = defaultdict(log_missing, current)
print(result)
print('Before:', dict(result))
for key, amount in increments:
    result[key] += amount
print('After: ', dict(result))

defaultdict(<function log_missing at 0x00000295E9318790>, {'green': 12, 'blue': 3})
Before: {'green': 12, 'blue': 3}
Key added
Key added
After:  {'green': 12, 'blue': 20, 'red': 5, 'orange': 9}


Supplying functions like log_missing makes APIs easy to build and test because it separates sied effects from deterministic behavior

In [7]:
def increment_with_report(current, increments):
    added_count = 0
    
    def missing():
        nonlocal added_count
        added_count += 1
        return 0
    
    result = defaultdict(missing, current)
    for key, amount in increments:
        result[key] += amount
    return result, added_count

result, count = increment_with_report(current, increments)
assert count == 2

### The problem with defining a closure for stateful hooks is that it's harder to read than the stateledd function example

In [8]:
class CountMissing:
    def __init__(self):
        self.added = 0
    
    def missing(self):
        self.added += 1
        return 0

In [9]:
counter = CountMissing()
result = defaultdict(counter.missing, current) # Method ref
for key, amount in increments:
    result[key]  += amount
assert counter.added == 2

### To clarify this situation, Python allows classes to define the __call__ special method.

* _ _call_ _ allows an object to be called just like a function.
* It also causes the callable built-in function to return True for such an instance

In [10]:
class BetterCountMissing:
    def __init__(self):
        self.added = 0
    
    def __call__(self):
        self.added += 1
        return 0
    
counter = BetterCountMissing()
assert counter() == 0
print(counter.added)
assert callable(counter)

1


In [11]:
counter = BetterCountMissing()
result = defaultdict(counter, current) # Relies on __call__
for key, amount in increments:
    result[key] += amount
assert counter.added == 2